In [ ]:
!pip3 install python-levenshtein

     |████████████████████████████████| 51kB 3.7MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144803 sha256=40011036aa11a31f438a09489c5923c798acdaea00a06eb8dd637deb364478e3
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pdb
import string
import pandas as pd
from Levenshtein import distance
import scipy
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import model_from_json
from keras.utils import to_categorical
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Load the model
model2 = tf.keras.models.load_model('/content/gdrive/MyDrive/EE698V/models/model_task1_40_65epochs_52')

In [ ]:
labl = ['air_conditioner','car_horn','children_playing','dog_bark','drilling','engine_idling','gun_shot','jackhammer','siren','street_music']

In [ ]:
# Load test files, loop over each, break into 40 frames, get predictions, do majority voting and check for consecutive 3 same labels, create output csv

final_test_path = '/content/gdrive/MyDrive/EE698V/test_task2/feats/'
save_path = '/content/gdrive/MyDrive/EE698V/test_task2/'
pred_file_name = []
pred_labels = []
for i in os.listdir(final_test_path):
  pred_file_name.append(i)
  test_image = np.load(final_test_path+i)
  iter = int(test_image.shape[1]/40)
  empty = []
  for j in range(iter):
    test_file = test_image[:,40*(j):40*(j+1)]
    test_file = test_file.reshape((1,)+test_file.shape + (1,))
    empty.append(np.argmax(model2.predict(test_file)))
  
  empty2 = []
  for count,pr in enumerate(empty):
    if count == 0 or count == len(empty)-1:
      continue
    if empty[count-1] == pr and empty[count+1] == pr:
      empty2.append(pr)

  empty3 = []
  empty3.append(empty2[0])
  for pr in empty2:
    if pr == empty3[-1]:
      continue
    else:
      empty3.append(pr)

  empty3 = np.array(empty3)

  final_string = ''
  for count,pr in enumerate(empty3):
    if count == empty3.size -1:
      final_string += labl[pr]
    else:
      final_string += labl[pr] + '-'

  pred_labels.append(final_string)

df = pd.DataFrame()
df['slice_file_name'] = pred_file_name
df['class'] = pred_labels
df.to_csv(save_path + 'task2_labels_test.csv')